In [1]:
import pandas as pd
import numpy as np
from statsmodels.tools.eval_measures import rmse

#from google.colab import drive

In [2]:
#drive.mount('/content/drive')

In [3]:
def df_split(df):
  categoria = []
  for x in df['category_name']:
    categoria.append(x.split('/'))
  return categoria


def sub_categoria(df):  
  lista = df_split(df)
  for y in range(len(lista)):
    lista[y] = lista[y][1]
  return lista

In [4]:
filtro_beauty = pd.read_csv('https://raw.githubusercontent.com/AugustoCRX/bootcamp_blue/main/novos_arquivos_treino/Limpeza%20v3/filtro_Beauty_train_v3.csv')

In [5]:
filtro_beauty.head()

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock,date2,day,month,year,description_clear,name_clear,brands_filter
0,Smashbox primer,2,Beauty/Makeup/Face,Smashbox,8.0,1,0.25 oz Full size is 1oz for [rm] in Sephora,20-8-2018,9,2018-08-20,20,8,2018,025 oz full size 1oz rm sephora,smashbox primer,smashbox
1,New vs pi k body mists,1,Beauty/Fragrance/Women,Victoria's Secret,34.0,0,(5) new vs pink body mists (2.5 oz each) Fresh...,4-9-2018,25,2018-04-09,9,4,2018,5 new v pink bodi mist 25 oz fresh clean sun k...,new v pi k bodi mist,victoria's secret
2,Sephora tarte birthday gift,1,Beauty/Makeup/Makeup Sets,Tarte,11.0,1,Brand new. Deluxe travel size products. Contai...,28-4-2018,9,2018-04-28,28,4,2018,brand new delux travel size product contain am...,sephora tart birthday gift,tarte
3,Glitter Eyeshadow,1,Beauty/Makeup/Eyes,Wet n Wild,6.0,1,2 glitter eyeshadows; one in Brass and one in ...,11-10-2018,10,2018-11-10,10,11,2018,2 glitter eyeshadow one brass one bleach,glitter eyeshadow,wet n wild
4,"Too Faced Limited ""Merry Macaroons""",1,Beauty/Makeup/Makeup Palettes,Too Faced,25.0,1,This AUTHENTIC pallete by Too Faced is brand n...,13-1-2018,30,2018-01-13,13,1,2018,authent pallet face brand new mint condit stil...,face limit `` merri macaroon '',too faced


In [6]:
filtro_beauty['sub_category_name'] = sub_categoria(filtro_beauty)

In [7]:
filtro_beauty.sub_category_name.value_counts()

Makeup                 47626
Skin Care              16182
Fragrance              13042
Tools & Accessories     8464
Hair Care               5764
Bath & Body             5451
Other                    366
Name: sub_category_name, dtype: int64

# Modelo 01

Regressão Linear sem restrição de preço e com variaveis dummies nas variaveis categoricas sub_category_name e brands_filter

In [8]:
import statsmodels.api as sm

In [9]:
filtro_beauty = pd.get_dummies(filtro_beauty, columns=['sub_category_name','brands_filter'])

In [10]:
X = filtro_beauty.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y = filtro_beauty.price
X = sm.add_constant(X)
rl = sm.OLS(exog=X,endog=y)
result = rl.fit()
print(result.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.396
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     722.5
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        14:28:33   Log-Likelihood:            -4.2765e+05
No. Observations:               96895   AIC:                         8.555e+05
Df Residuals:                   96806   BIC:                         8.563e+05
Df Model:                          88                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [11]:
filtro_beauty['price'].describe()

count    96895.000000
mean        23.320409
std         25.714817
min          3.000000
25%         10.000000
50%         17.000000
75%         28.000000
max       1315.000000
Name: price, dtype: float64

In [12]:
m1_ypred = result.predict(X)

In [13]:
m1_RMSE = rmse(y,m1_ypred)
m1_RMSE.round(2)

19.98

# Modelo 02

Mantem caracteristicas do modelo 1 com filtro de preço ate $150,00

In [14]:
filtro_beauty_ajustado = pd.read_csv('https://raw.githubusercontent.com/AugustoCRX/bootcamp_blue/main/novos_arquivos_treino/Limpeza%20v3/filtro_Beauty_train_v3.csv')
filtro_beauty_ajustado['sub_category_name'] = sub_categoria(filtro_beauty_ajustado)
filtro_beauty_ajustado = pd.get_dummies(filtro_beauty_ajustado, columns=['sub_category_name','brands_filter'])

In [15]:
filtro_beauty_ajustado = filtro_beauty_ajustado[filtro_beauty_ajustado['price'] <= 150]

In [16]:
X1 = filtro_beauty_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y1 = filtro_beauty_ajustado.price
X1 = sm.add_constant(X1)
rl1 = sm.OLS(exog=X1,endog=y1)
result1 = rl1.fit()
print(result1.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.526
Model:                            OLS   Adj. R-squared:                  0.526
Method:                 Least Squares   F-statistic:                     1229.
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        14:28:37   Log-Likelihood:            -3.8166e+05
No. Observations:               96434   AIC:                         7.635e+05
Df Residuals:                   96346   BIC:                         7.643e+05
Df Model:                          87                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [17]:
filtro_beauty_ajustado['price'].describe()

count    96434.000000
mean        22.268157
std         18.397502
min          3.000000
25%         10.000000
50%         17.000000
75%         28.000000
max        150.000000
Name: price, dtype: float64

In [18]:
m2_ypred = result1.predict(X1)
m2_RMSE = rmse(y1,m2_ypred)
m2_RMSE.round(2)

12.66

# Modelo 03

Arvore de Regressão usando como criterio de seleção dos nós a métrica R²

In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [20]:
X2 = filtro_beauty.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y2 = filtro_beauty.price

In [21]:
m3_dtr = DecisionTreeRegressor(random_state=10)
result2 = m3_dtr.fit(X2,y2)

In [22]:
m3_r2 = result2.score(X2,y2)
m3_ypred = result2.predict(X2)
m3_mse = mean_squared_error(y2,m3_ypred)
m3_rmse = m3_mse**0.5

In [23]:
print(f"R² = {m3_r2.round(2)}")
print(f"MSE = {m3_mse.round(2)}")
print(f"RMSE = {m3_rmse.round(2)}")


R² = 0.42
MSE = 383.06
RMSE = 19.57


# Modelo 04

Arvore de Regressão usando como criterio de seleção dos nós a métrica R² e com filtro de preço ate $150,00

In [24]:
X3 = filtro_beauty_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y3 = filtro_beauty_ajustado.price

In [25]:
m4_dtr = DecisionTreeRegressor(random_state=10)
result3 = m4_dtr.fit(X3,y3)

In [26]:
m4_r2 = result3.score(X3,y3)
m4_ypred = result3.predict(X3)
m4_mse = mean_squared_error(y3,m4_ypred)
m4_rmse = m4_mse**0.5

In [27]:
print(f"R² = {m4_r2.round(2)}")
print(f"MSE = {m4_mse.round(2)}")
print(f"RMSE = {m4_rmse.round(2)}")

R² = 0.56
MSE = 150.22
RMSE = 12.26


# Resultados

| Modelo | R² |RMSE | Observação |
| --- | --- | --- | --- |
| Modelo 01 | 39,6% | 19,98 | Regressão Linear |
| Modelo 02 | 52,6% | 12,66 | Regressão Linear (preço <= 150) |
| Modelo 03 | 42,0% | 19,57 | Arvore Regressão (criterio = R²) |
| Modelo 04 | 56,0% | 12,26 | Arvore Regressão (criterio = R², preço <= 150) |